## Tinkering with NLB API
14th August 2022

In [4]:
# !pip3 install zeep

In [10]:
import pandas as pd
from datetime import datetime
from zeep import Client, helpers

### Resources
These are the resources that I referred to
1. [NLB API documentation](https://opendata.nlb.gov.sg/content/SkillsFuture/NLB_Labs_TechDoc-V3.6.pdf)
1. [Unofficial NLB API Python Github repo](https://github.com/yi-jiayu/nlbsg) from [Jiayu](https://blog.jiayu.co/) - recommended by the NLB API documentation.

### Authentication

In [11]:
API = pd.read_csv("api.csv")['api'].values[0]
PRODUCTION_URL = "https://openweb.nlb.gov.sg/OWS/CatalogueService.svc?singleWsdl"
client = Client(wsdl=PRODUCTION_URL)

### Content map
1. Show how the input looks like for Search
1. Show how basic Search output looks like, and explain its structure
1. Show how this works for both the other two methods.
1. Show how Title recommendations don't seem to work - And confirm that I am emailing NLB on this
1. Write a ReadMe to explain how to use the API

### Catalogue Endpoint
Using the [Zeep](https://docs.python-zeep.org/en/master/) Python package meant that I didn't need to handle tricky XML formats, and could just nested dictionary as inputs for my NLB API calls. Below are some sample API calls that I made.

### 1st Method - Search Feature
The following is the `input structure` needed to make the API call for the Search method. 

In [4]:
search_input = {
    "APIKey": API,
    "SearchItems": {
        "SearchItem": [
            {
                "SearchField": "Title", 
                "SearchTerms": "Robust Python"
            },
            {
                "SearchField": "Keywords", 
                "SearchTerms": "Python"
            },
        ]
    },
    "Modifiers": {
        "SortSchema": None,
        "StartRecordPosition": 1,
        "MaximumRecords": 100,
        "SetId": None
    }
}

There are 3 major input sections for the Search method - (1) API (2) SearchItems (3) Modifiers.

1. `API key` - This is what you get when you request for the API key from NLB.
1. Input parameters for `SearchItems`. If you want to have more than one `SearchItems` parameter, you need to include them as a dictionary of `SearchField` and `SearchTerms`. 
    1. Keywords
    1. Author
    1. Subject
    1. Title
    1. Branch
    1. Media_code
    1. Language
1. `Modifiers` don't directly affect the results being returned, but it does allow one to modify how the results are shown. Their input parameters are:
    1. SortSchema
    1. StartRecordPosition
    1. MaximumRecords
    1. SetId

`zeep` seems to require me to add `**` to my `client.service.Search()` input for the API to work. I also used `helpers.serialize_object` to convert the API output into an OrderDict for further processing.

In [5]:
test_output = helpers.serialize_object(
    client.service.Search(**search_input)
)

This is how my output looks like 

In [6]:
test_output

OrderedDict([('Status', 'OK'),
             ('Message', 'Operation completed successfully'),
             ('ErrorMessage', None),
             ('TotalRecords', 2),
             ('NextRecordPosition', 0),
             ('SetId', '367795457'),
             ('Titles',
              OrderedDict([('Title',
                            [OrderedDict([('BID', '205485970'),
                                          ('ISBN',
                                           '9781098100636 (electronic bk)|9781098100612 (electronic bk)'),
                                          ('TitleName',
                                           'Robust python. Patrick Viafore.'),
                                          ('Author', 'Viafore, Patrick.'),
                                          ('PublishYear', '2021'),
                                          ('MediaCode', 'BK'),
                                          ('MediaDesc', 'Books')]),
                             OrderedDict([('BID', '205499632'),
    

In [7]:
test_output.get('TotalRecords')

2

One rather irritating thing about the Search method is, I wasn't able to find a way to separate physical and electronic books, even when I have put the parameter `MediaDesc` to be `Books`. This would be a bit troublesome if I only wanted to look at physical books (which actually is my most common use case).

### Small issue with MaximumRecords being no more than 50...

In [8]:
search_input = {
    "APIKey": API,
    "SearchItems": {
        "SearchItem": [
            {
                "SearchField": "Title", 
                "SearchTerms": "Python"
            },
        ]
    },
    "Modifiers": {
        "SortSchema": None,
        "StartRecordPosition": 1,
        "MaximumRecords": 100,
        "SetId": None
    }
}

test_output = helpers.serialize_object(
    client.service.Search(**search_input)
)

# Here, I am pulling out all the titles from the API, and counting how many records I have. 
# Without paginating across all the 771 records, I only have 50 records.
len(test_output.get("Titles").get("Title"))

50

In [9]:
test_output.get('TotalRecords')

772

Interestingly, the [official NLB documentation](https://opendata.nlb.gov.sg/content/SkillsFuture/NLB_Labs_TechDoc-V3.6.pdf) mentions that the maximum records that I can get at any one time is 100, but I was only able to get a maximum of 50 records per section of the API. This is as of my time of writing at DD-MM-YYYY.

### 2nd Method - Available Titles by Library 
This method is useful in finding the available books in the NLB, and this even includes electronic books! **However, I have to either give the ISBN or BID number**. I cannot make this API call through any other parameters. Unfortunately, there is a very low chance I will outright have these numbers, so most probably it means I need to make an API on the NLB Search feature, extract the ISBN / BID number from there, and then use that as an input parameter to make this API call. 

In [15]:
# bid_no = 9781101595954
# bid_no = 205431306
isbn = 9781492057697

get_avail = {
    "APIKey": API,
    "ISBN": isbn,
    # "BID": bid_no,
    "Modifiers" : {
        "SortSchema": None,
        "StartRecordPosition": 1,
        "MaximumRecords": 100,
        "SetId": None
    },
}

In [18]:
avail_info = client.service.GetAvailabilityInfo(**get_avail)
avail_info

{
    'Status': 'OK',
    'Message': 'Operation completed successfully',
    'ErrorMessage': None,
    'NextRecordPosition': 0,
    'SetId': '369196980',
    'Items': {
        'Item': [
            {
                'ItemNo': 'B36549693A',
                'BranchID': 'QUPL',
                'BranchName': 'Queenstown Public Library',
                'LocationCode': '____',
                'LocationDesc': 'Adult Lending',
                'CallNumber': 'English 005.133 GIF -[COM]',
                'StatusCode': 'S',
                'StatusDesc': 'Not on Loan',
                'MediaCode': 'BOOK',
                'MediaDesc': 'Book',
                'StatusDate': '07/06/2022',
                'DueDate': None,
                'ClusterName': None,
                'CategoryName': None,
                'CollectionCode': 'GGEN',
                'CollectionMinAgeLimit': None
            },
            {
                'ItemNo': 'B36549692K',
                'BranchID': 'TRL',
                '

To make the output easier to read, I wrote some light Panda code to reshape the OrderDict into a pandas dataframe. 

In [17]:
df = pd.DataFrame()
for i in pd.DataFrame(helpers.serialize_object(avail_info).get("Items").values()).T[0]:
    df = df.append(pd.DataFrame.from_dict(i, orient='index').T)
df['isbn'] = isbn
df.sort_values("BranchName")[["BranchName", "StatusDesc", "DueDate"]]

,BranchName,StatusDesc,DueDate
0,Ang Mo Kio Public Library,Not on Loan,None
0,Bedok Public Library,Not on Loan,None
0,Bishan Public Library,Not on Loan,None
0,Geylang East Public Library,On Loan,22/08/2022
0,Jurong Regional Library,On Loan,30/09/2022
0,Jurong West Public Library,Not on Loan,None
0,Queenstown Public Library,Not on Loan,None
0,Tampines Regional Library,On Loan,24/09/2022
0,The LLiBrary (Lifelong Learning Institute),On Loan,03/09/2022
0,Toa Payoh Public Library,In-Transit,None


### 3rd Method - Get Title Details 
Similar to the `GetAvailabilityInfo` method, this method also only allows either ISBN or BID as input parameters. 

In [6]:
title_inputs = {
    "APIKey": API,
    "BID": 204485571,
}

In [7]:
title_details = client.service.GetTitleDetails(**title_inputs)
title_details

{
    'Status': 'OK',
    'Message': 'Operation completed successfully',
    'ErrorMessage': None,
    'TitleDetail': {
        'BID': '204485571',
        'TitleName': '40 algorithms every programmer should know : hone your problem-solving skills by learning different algorithms and their implementation in Python / Imran Ahmad.',
        'Author': 'Ahmad, Imran,',
        'OtherAuthors': None,
        'Publisher': None,
        'PhysicalDesc': 'x, 365 pages :illustrations ;24 cm',
        'Subjects': {
            'Subject': [
                'Computer programs',
                'Python (Computer program language)'
            ]
        },
        'Summary': None,
        'Notes': '\r\nIncludes index.',
        'ISBN': '1789801214 (paperback)|9781789801217 (paperback)',
        'ISSN': None,
        'NTitleName': None,
        'NAuthor': None,
        'NPublisher': None
    }
}

### Read Recommendations
From their documentation - `Read Alike Service provides search of related titles to the one submitted in the request message. It is also known Title Recommendation Service.` I finally figured out how to make the API call, but the error message is that I have no access rights to it...

In [ ]:
get_recom = {
    "APIKey": API,
    "BIDS": {
        "BID": 204485571
    },
    "BidType" : "NLB"
}

PRODUCTION_URL = "http://openweb.nlb.gov.sg/OWS/ReadAlikeService.svc?singleWsdl"
client = Client(wsdl=PRODUCTION_URL)

get_recommendations = client.service.GetRecommendationsForTitles(**get_recom)
get_recommendations

In [ ]:
PRODUCTION_URL = "https://openweb.nlb.gov.sg/OWS/EResourceService.svc?singleWsdl"
catalogue_client = Client(wsdl=PRODUCTION_URL)
dir(catalogue_client.service)

#### eResource API 

In [38]:
4713 + 829

5542

In [40]:
PRODUCTION_URL = "https://openweb.nlb.gov.sg/OWS/EresourceService.svc?wsdl"
client = Client(wsdl=PRODUCTION_URL)

ebook_search = {
    "APIKey": API,
    "SearchItems": {
        "SearchItem": [
            {
                "SearchField": "Title", 
                "SearchTerms": "News"
            }
        ]
    },
    "ContentType": 'Digital Books',
    "DataFrom": "Netlibrary",
    "Modifiers": {
        "SortSchema": None,
        "StartRecordPosition": 1,
        "MaximumRecords": 1
    }
}

ebook_search_output = client.service.Search(**ebook_search)
ebook_search_output

{
    'Status': 'OK',
    'Message': 'Operation completed successfully',
    'ErrorMessage': None,
    'TotalRecords': 695,
    'NextRecordPosition': 2,
    'Results': {
        'Result': [
            {
                'ID': 'fc261a69-0d29-4d7b-b2ed-6125de192692',
                'Types': {
                    'Value': [
                        'Electronic Book'
                    ]
                },
                'Title': 'Malaysia [electronic resource] : New States in a New Nation. ',
                'Author': 'Milne, R.S.',
                'Abstracts': {
                    'Value': [
                        'First Published in 1974. Routledge is an imprint of Taylor & Francis, an informa company. '
                    ]
                },
                'Languages': {
                    'Value': [
                        'eng'
                    ]
                },
                'CreationDate': '01/01/2014 00:00:00',
                'DataFrom': 'Netlibrary',
            